In [48]:
import psycopg2
import os
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
import re
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
%matplotlib inline
os.chdir('/home/ubuntu/F/Jagadeesh/rdsfiles')
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
import datetime
from datetime import date, timedelta
import pyreadr

In [49]:
result = pyreadr.read_r('bv_property_updated.rds')
result = result[None]
Final_data= pyreadr.read_r('Final_data_bookings.rds')
#Final_data= pyreadr.read_r('Latest_data.rds')
Final_data=Final_data[None]

In [50]:
Final_data11=Final_data.copy()

In [51]:
# Final_data.head()
Final_data=Final_data11.copy()
import datetime as dt
Final_data['bookingdate']=pd.TimedeltaIndex(Final_data['bookingdate'],unit='d')+dt.datetime(1970,1,1)
Final_data['arrivaldate']=pd.TimedeltaIndex(Final_data['arrivaldate'],unit='d')+dt.datetime(1970,1,1)
Final_data['date']=pd.TimedeltaIndex(Final_data['date'],unit='d')+dt.datetime(1970,1,1)
Final_data['departuredate']=pd.TimedeltaIndex(Final_data['departuredate'],unit='d')+dt.datetime(1970,1,1)
Final_data=Final_data[['date','house_id','bookingdate', 'arrivaldate', 'departuredate','channel_type','number_of_adults', 'number_of_children', 'number_of_babies','number_of_pets','gross_booking_value']]

In [52]:
result=result[~result.cluster.isna()]
valuable_wk_result=result.groupby(['cluster','week'])[['valuable_wk','bedroom_count','number_of_persons']].sum().reset_index()
# for i in range(0,52):
#     print(result[result.valuable_wk==0])

Final_data=Final_data[~Final_data.date.isna()]
Final_data['week']=pd.to_numeric(Final_data.date.dt.week.astype('int64'))
Final_data['year']=pd.to_numeric(Final_data.date.dt.year.astype('int64'))
# Final_data_new=pd.merge(Final_data,result[['house_id','cluster','hol_period','week','country_name']],how='left',on=['house_id','week'])
Final_data_new=pd.merge(Final_data,result[['house_id','cluster','hol_period','week']],how='left',on=['house_id','week'])
Final_data_new['lead_day']=Final_data_new['arrivaldate']-Final_data_new['bookingdate']
Final_data_new['no_days']=Final_data_new['departuredate']-Final_data_new['arrivaldate']
Final_data_new['gross_booking_value']=Final_data_new['gross_booking_value']/Final_data_new['no_days'].dt.days

In [54]:
#exclusive_data = result.groupby('cluster')['exclusive_12'].value_counts().unstack().reset_index().fillna(0)
exclusive_data = result[['house_id','cluster','exclusive']].drop_duplicates()

# Exclusive data
exclusive_data = exclusive_data.groupby('cluster')['exclusive'].value_counts().unstack().reset_index().fillna(0)
exclusive_data = exclusive_data[['cluster','Exclusive','non-Exclusive']] 
exclusive_data['exclusive_perc']=pd.to_numeric(exclusive_data['Exclusive']/(exclusive_data['Exclusive']+exclusive_data['non-Exclusive']))
exclusive_data['non-exclusive_perc']=pd.to_numeric(exclusive_data['non-Exclusive']/(exclusive_data['Exclusive']+exclusive_data['non-Exclusive']))
exclusive_data = exclusive_data[['cluster','exclusive_perc','non-exclusive_perc']] 



In [55]:
pets_allowed_data = result[['house_id','cluster','pets_allowed']].drop_duplicates()
pets_allowed_data = pets_allowed_data.groupby('cluster')['pets_allowed'].value_counts().unstack().reset_index().fillna(0)
pets_allowed_data =  pets_allowed_data[['cluster','N','Y']] 
#pets_allowed_data.describe()
pets_allowed_data['N_perc']=pd.to_numeric(pets_allowed_data['N']/(pets_allowed_data['N']+pets_allowed_data['Y']))
pets_allowed_data['Y_perc']=pd.to_numeric(pets_allowed_data['Y']/(pets_allowed_data['N']+pets_allowed_data['Y']))
pets_allowed_data =  pets_allowed_data[['cluster','N_perc','Y_perc']] 


In [56]:
# accomodation_type

acco_data = result[['house_id','cluster','accommodation_type']].drop_duplicates()
acco_data = acco_data.groupby('cluster')['accommodation_type'].value_counts().unstack().reset_index().fillna(0)
acco_data =  acco_data[['cluster','Apartment','Boat','House','Tentlodge','Unknown']] 
acco_data['Apartment_perc'] = pd.to_numeric(acco_data['Apartment']/(acco_data[['cluster','Apartment','Boat','House','Tentlodge','Unknown']].sum(axis=1))) 
acco_data['Boat_perc'] = pd.to_numeric(acco_data['Boat']/(acco_data[['cluster','Apartment','Boat','House','Tentlodge','Unknown']].sum(axis=1))) 
acco_data['House_perc'] = pd.to_numeric(acco_data['House']/(acco_data[['cluster','Apartment','Boat','House','Tentlodge','Unknown']].sum(axis=1))) 
acco_data['Tentlodge_perc'] = pd.to_numeric(acco_data['Tentlodge']/(acco_data[['cluster','Apartment','Boat','House','Tentlodge','Unknown']].sum(axis=1))) 
acco_data['Unknown_perc'] = pd.to_numeric(acco_data['Unknown']/(acco_data[['cluster','Apartment','Boat','House','Tentlodge','Unknown']].sum(axis=1))) 
acco_data = acco_data[['cluster','Apartment_perc','Boat_perc','House_perc','Tentlodge_perc','Unknown_perc']]
#acco_data.columns

In [57]:
Final_data_new['urn']=1
Final_data_new=Final_data_new[Final_data_new.arrivaldate<'2019-05-01']
#Final_data_new['date_unique']=
Final_data_new['date_unique']=(Final_data_new['year'].astype('str')+Final_data_new['week'].astype('str')).astype('int64')
# Final_data_new['date_unique']=train_data['date_unique'].astype('int64')
Urn_cluster=Final_data_new.groupby(['year','week','cluster','date_unique'])['urn'].sum().reset_index()

forecasting_weeks=9
predict_log=(forecasting_weeks*7)
Final_data_new['lead_day']=Final_data_new['lead_day'].dt.days
Final_data_new['Indirect']=0
Final_data_new['Direct']=0
Final_data_new.loc[Final_data_new.channel_type=='Indirect','Indirect']=1
Final_data_new.loc[Final_data_new.channel_type=='Direct','Direct']=1
temp_df=Final_data_new[Final_data_new['lead_day']>=predict_log]

In [ ]:
#len(Urn_cluster[Urn_cluster.urn.isna()])
#Final_data_new[(Final_data_new.cluster==179)& (Final_data_new.week==18)  & (Final_data_new.year==2019) ]
# len(Urn_cluster)
# Urn_cluster.head()

In [58]:
temp_created = Final_data.copy()
temp_created['lead_day']=temp_created['arrivaldate']-temp_created['bookingdate']
temp_created['lead_day'] = temp_created['lead_day'].dt.days
temp_created['lead_day']=temp_created.lead_day.astype('int64')
temp_created = temp_created[temp_created['lead_day']>=predict_log]
temp_created=pd.merge(temp_created,result[['house_id','cluster','hol_period','week']],how='left',on=['house_id','week'])
# temp_created.head(20)

In [59]:
def brn_date(tf,week_bck,week_fwd):
    year=tf['year']
    week=tf['week']
    weeks=week-week_bck+week_fwd
    if weeks<=0:
        weeks=52+weeks
        year=year-1
    if weeks>52:
        weeks=weeks-52
        year=year+1
#     print(weeks,year)
    return weeks,year

### function which returns the prebooked_URNS for the adjacent weeks 
def bre_leadday(df,predict_log,week_bck=0,week_fwd=0):
    predict_log_new=predict_log+(-week_fwd*7)+(week_bck*7)
    print(predict_log_new)
    temp=df[df['lead_day']>=predict_log_new]
    temp=temp.groupby(['year','week','cluster'])['urn'].sum().reset_index()
    temp[['week','year']]=temp.apply(brn_date, axis=1, result_type="expand",args=(week_bck,week_fwd,))
    pre_urnname='pre_urn_Wb_'+str(week_bck)+'_WF_'+str(week_fwd)
    temp.rename(columns={'urn':pre_urnname},inplace=True)
    return temp

In [60]:
#getting previus weeks and coming weeks urns
pre_brn_1wback=bre_leadday(Final_data_new,predict_log,week_bck=0,week_fwd=1)
pre_brn_1wfwd=bre_leadday(Final_data_new,predict_log,week_bck=1,week_fwd=0)
pre_brn_2wback=bre_leadday(Final_data_new,predict_log,week_bck=0,week_fwd=2)
pre_brn_2wfwd=bre_leadday(Final_data_new,predict_log,week_bck=2,week_fwd=0)

56
70
49
77


In [61]:
def q1(x):
    return x.quantile(0.25)

def q2(x):
    return x.quantile(0.50)

def q3(x):
    return x.quantile(0.75)

f = {'lead_day': [q1,q2,q3]}
temp_created_1=temp_created[['cluster','week','year','lead_day']].groupby(['cluster','week','year']).agg(f).reset_index()

In [62]:
temp_df['ppl']=temp_df[['number_of_adults', 'number_of_children','number_of_babies']].sum(axis=1)

# temp_df=temp_df[['cluster','week','year','urn','country_name','hol_period','ppl','Direct','Indirect','number_of_adults', 'number_of_children','number_of_babies', 'number_of_pets','gross_booking_value']]
# train_data=temp_df.groupby(['cluster','week','year','country_name','hol_period'])[['urn','ppl','Direct','Indirect','number_of_adults', 'number_of_children','number_of_babies', 'number_of_pets','gross_booking_value']].sum().reset_index()
# merger_data=temp_df.groupby(['cluster','week','year','country_name','hol_period'])[['number_of_adults', 'number_of_children','number_of_babies', 'number_of_pets']].sum().reset_index()
# train_data=pd.merge(train_data,merger_data[['cluster','week','year','number_of_adults', 'number_of_children','number_of_babies', 'number_of_pets']],how='left',on=['cluster','week','year'])

temp_df=temp_df[['cluster','week','year','urn','hol_period','ppl','Direct','Indirect','number_of_adults', 'number_of_children','number_of_babies', 'number_of_pets','gross_booking_value']]
train_data=temp_df.groupby(['cluster','week','year','hol_period'])[['urn','ppl','Direct','Indirect','number_of_adults', 'number_of_children','number_of_babies', 'number_of_pets','gross_booking_value']].sum().reset_index()

# merger_data=temp_df.groupby(['cluster','week','year','hol_period'])[['number_of_adults', 'number_of_children','number_of_babies', 'number_of_pets']].sum().reset_index()
# train_data=pd.merge(train_data,merger_data[['cluster','week','year','number_of_adults','number_of_babies', 'number_of_pets']],how='left',on=['cluster','week','year'])
# train_data=pd.merge(train_data,merger_data[['cluster','week','year','number_of_adults', 'number_of_children','number_of_babies', 'number_of_pets']],how='left',on=['cluster','week','year'])
# train_data_cl=train_data.copy()
# train_data = pd.merge(train_data,temp_created_1,how='left',on=['cluster','week','year'])

#train_data=train_data_cl.copy()

In [63]:
properties_data = result[['house_id','cluster']].drop_duplicates()
prop_count = properties_data.cluster.value_counts().reset_index()
prop_count.columns = ['cluster','properties_count']
prop_count.head()

,cluster,properties_count
0,60,1300
1,191,1106
2,207,836
3,52,806
4,106,767


In [64]:
def get_data(train_data,year,week,date_unique,year_ma=3,week_ma=1,week_fw=0):
    yrss = [yrs for yrs in range(year-year_ma,year)]
    weeks=[-(52+wk) if wk <=0 else wk for wk in range(week-week_ma,week+1+week_fw)]
    df=pd.DataFrame()
    for a in yrss:
        for b in weeks:
            if b<0:
                b1,a1=-(b),a-1
            elif b>52:
                b1,a1=b-52,a+1
            else:
                b1,a1=b,a
            temp=train_data[(train_data.year==a1) & (train_data.week==b1)][['cluster','urn']]
            df=pd.concat([temp,df])
    df.dropna(inplace=True)
    df=df.groupby('cluster').urn.mean().reset_index()
    df['date_unique']=int(date_unique)
    mean_urn_str='pre_mean_urn_y_'+str(year_ma)+'_w_'+str(week_ma)+'f'+str(week_fw)
    df.rename(columns={'urn':mean_urn_str},inplace=True)
    return df  

In [65]:
cluste_meandf=pd.DataFrame()
tr_yrs=list(train_data.year.unique())
cluste_mean_y_2_w_0=pd.DataFrame()
cluste_mean_y_2_w_2=pd.DataFrame()
cluste_mean_y_2_w_1_f_1=pd.DataFrame()
for yr in tr_yrs:
    for wk in list(range(1,53)):
        date_unique=str(yr)+str(wk)
        rdf=get_data(train_data,yr,wk,date_unique)
        cluste_meandf=pd.concat([cluste_meandf,rdf])
        rdf_y_2_w_0=get_data(train_data,yr,wk,date_unique,2,0)
        cluste_mean_y_2_w_0=pd.concat([cluste_mean_y_2_w_0,rdf_y_2_w_0])
        rdf_y_2_w_2=get_data(train_data,yr,wk,date_unique,2,2)
        cluste_mean_y_2_w_2=pd.concat([cluste_mean_y_2_w_2,rdf_y_2_w_2])
        rdf_y_2_w_1_f_1=get_data(train_data,yr,wk,date_unique,2,2,1)
        cluste_mean_y_2_w_1_f_1=pd.concat([cluste_mean_y_2_w_1_f_1,rdf_y_2_w_1_f_1])

In [67]:
train_data['date_unique']=train_data['year'].astype('str')+train_data['week'].astype('str')
train_data['date_unique']=train_data['date_unique'].astype('int64')

In [68]:
# cluste_meandf.rename(columns={'mean_urn_y_3_w_1':'pre_mean_urn_y_3_w_1'},inplace=True)
# #cluste_meandf.head()
train_data.head()
# #cluste_mean_y_2_w_0.head()
# cluste_mean_y_2_w_0.rename(columns={'mean_urn_y_2_w_0':'pre_mean_urn_y_2_w_0'},inplace=True)
# #cluste_mean_y_2_w_2.head()
# cluste_mean_y_2_w_0.rename(columns={'mean_urn_y_2_w_2':'pre_mean_urn_y_2_w_2'},inplace=True)



,cluster,week,year,hol_period,urn,ppl,Direct,Indirect,number_of_adults,number_of_babies,number_of_pets,gross_booking_value,date_unique
0,0,1,2012,Christmas,4,36.0,2,2,36,0,0,1771.571429,20121
1,0,1,2013,Christmas,32,296.0,10,22,270,4,0,12632.000000,20131
2,0,1,2014,Christmas,37,341.0,5,32,292,8,12,14011.928571,20141
3,0,1,2015,Christmas,12,92.0,2,10,86,0,8,4665.714286,20151
4,0,1,2016,Christmas,24,151.0,0,24,151,0,0,7216.928571,20161


In [69]:
train_data.rename(columns={'urn':'pre_urn'},inplace=True)
train_data=pd.merge(train_data,Urn_cluster,how='left',on=['cluster','week','year','date_unique'])

# train_data['per_pre']=(train_data['urn']-train_data['pre_urn'])/train_data['urn']
# train_data[train_data.urn>100][['cluster','week','year','pre_urn','urn','per_pre']]
# np.corrcoef(train_data['pre_urn'],train_data['urn'])

In [71]:
#len(train_data[train_data.pre_urn.isna()])/len(train_data)
train_data=pd.merge(train_data,cluste_meandf,how='left',on=['cluster','date_unique'])
train_data=pd.merge(train_data,cluste_mean_y_2_w_0,how='left',on=['cluster','date_unique'])
train_data=pd.merge(train_data,cluste_mean_y_2_w_2,how='left',on=['cluster','date_unique'])
train_data=pd.merge(train_data,cluste_mean_y_2_w_1_f_1,how='left',on=['cluster','date_unique'])
train_data=pd.merge(train_data,prop_count,how='left',on=['cluster'])


#train_data.columns

In [72]:
#merging pre and post weeks pre_urns
train_data=pd.merge(train_data,pre_brn_1wback,how='left',on=['cluster','week','year'])
train_data=pd.merge(train_data,pre_brn_1wfwd,how='left',on=['cluster','week','year'])
train_data=pd.merge(train_data,pre_brn_2wback,how='left',on=['cluster','week','year'])
train_data=pd.merge(train_data,pre_brn_2wfwd,how='left',on=['cluster','week','year'])
# preclu=[cl for cl in list(train_data.columns) if cl.startswith('pre_urn_W')]
# train_data.loc[:,preclu]=train_data.loc[:,preclu].fillna(0)

In [73]:
train_data[train_data.pre_urn.isna()].describe()
#train_data.describe()
#train_data[train_data.pre_urn_Wb_0_WF_1==277]
#train_data['date_unique']=train_data['year'].astype('str')+train_data['week'].astype('str')
#train_data['date_unique']=train_data['date_unique'].astype('int64')

,cluster,week,year,pre_urn,ppl,Direct,Indirect,number_of_adults,number_of_babies,number_of_pets,gross_booking_value,date_unique,urn,pre_mean_urn_y_3_w_1f0,pre_mean_urn_y_2_w_0f0,pre_mean_urn_y_2_w_2f0,pre_mean_urn_y_2_w_2f1,properties_count,pre_urn_Wb_0_WF_1,pre_urn_Wb_1_WF_0,pre_urn_Wb_0_WF_2,pre_urn_Wb_2_WF_0
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# valuable_wk_result.head()


In [74]:
from sklearn import preprocessing
os.chdir('/home/ubuntu/F/Jagadeesh/tdata/pick')
le = preprocessing.LabelEncoder()
dtyps=train_data.dtypes.reset_index().rename({0:'type','index':'clumn'},axis=1)
train_datacolumns=dtyps.loc[dtyps['type']=='object'].clumn.to_list()
for column_name in train_datacolumns:
    try:
        le.fit(train_data[column_name])
        print(column_name)
        train_data[column_name] = le.fit_transform(train_data[column_name])

    except Exception as e:
        print(e,column_name)
        train_data[column_name]=train_data[column_name].factorize()[0]
        train_data[column_name] = le.fit_transform(train_data[column_name])

hol_period


In [75]:
train_data.hol_period.unique()

array([ 3,  7, 11,  4, 12,  5, 10, 13, 14,  1,  2,  8,  0,  6, 15,  9])

In [76]:
def get_data(train_data,year,week,date_unique,year_ma=3,week_ma=1,week_fw=0):
    yrss = [yrs for yrs in range(year-year_ma,year)]
    weeks=[-(52+wk) if wk <=0 else wk for wk in range(week-week_ma,week+1+week_fw)]
    df=pd.DataFrame()
    for a in yrss:
        for b in weeks:
            if b<0:
                b1,a1=-(b),a-1
            elif b>52:
                b1,a1=b-52,a+1
            else:
                b1,a1=b,a
            temp=train_data[(train_data.year==a1) & (train_data.week==b1)][['cluster','urn']]
            df=pd.concat([temp,df])
    df.dropna(inplace=True)
    df=df.groupby('cluster').urn.mean().reset_index()
    df['date_unique']=int(date_unique)
    mean_urn_str='mean_urn_y_'+str(year_ma)+'_w_'+str(week_ma)+'f'+str(week_fw)
    df.rename(columns={'urn':mean_urn_str},inplace=True)
    return df  

In [77]:
cluste_meandf=pd.DataFrame()
tr_yrs=list(train_data.year.unique())
cluste_mean_y_2_w_0=pd.DataFrame()
cluste_mean_y_2_w_2=pd.DataFrame()
cluste_mean_y_2_w_1_f_1=pd.DataFrame()
for yr in tr_yrs:
    for wk in list(range(1,53)):
        date_unique=str(yr)+str(wk)
        rdf=get_data(train_data,yr,wk,date_unique)
        cluste_meandf=pd.concat([cluste_meandf,rdf])
        rdf_y_2_w_0=get_data(train_data,yr,wk,date_unique,2,0)
        cluste_mean_y_2_w_0=pd.concat([cluste_mean_y_2_w_0,rdf_y_2_w_0])
        rdf_y_2_w_2=get_data(train_data,yr,wk,date_unique,2,2)
        cluste_mean_y_2_w_2=pd.concat([cluste_mean_y_2_w_2,rdf_y_2_w_2])
        rdf_y_2_w_1_f_1=get_data(train_data,yr,wk,date_unique,2,2,1)
        cluste_mean_y_2_w_1_f_1=pd.concat([cluste_mean_y_2_w_1_f_1,rdf_y_2_w_1_f_1])

In [78]:
preclu=[cl for cl in list(train_data.columns) if cl.startswith('pre_mean_urn')]
train_data.loc[:,preclu]=train_data.loc[:,preclu].fillna(0)

In [79]:
train_data=pd.merge(train_data,cluste_meandf,how='left',on=['cluster','date_unique'])
train_data=pd.merge(train_data,cluste_mean_y_2_w_0,how='left',on=['cluster','date_unique'])
train_data=pd.merge(train_data,cluste_mean_y_2_w_2,how='left',on=['cluster','date_unique'])
train_data=pd.merge(train_data,cluste_mean_y_2_w_1_f_1,how='left',on=['cluster','date_unique'])


In [80]:
#train_data.head()
train_data=pd.merge(train_data,valuable_wk_result,how='left',on=['cluster','week'])
train_data = pd.merge(train_data,exclusive_data,how='left',on=['cluster'])
train_data = pd.merge(train_data,pets_allowed_data,how='left',on=['cluster'])
train_data = pd.merge(train_data,acco_data,how='left',on=['cluster'])


In [81]:
# train_data.isnull().sum()
train_data=pd.merge(train_data,prop_count,how='left',on=['cluster'])


In [83]:
train_data.columns

Index(['cluster', 'week', 'year', 'hol_period', 'pre_urn', 'ppl', 'Direct',
       'Indirect', 'number_of_adults', 'number_of_babies', 'number_of_pets',
       'gross_booking_value', 'date_unique', 'urn', 'pre_mean_urn_y_3_w_1f0',
       'pre_mean_urn_y_2_w_0f0', 'pre_mean_urn_y_2_w_2f0',
       'pre_mean_urn_y_2_w_2f1', 'properties_count_x', 'pre_urn_Wb_0_WF_1',
       'pre_urn_Wb_1_WF_0', 'pre_urn_Wb_0_WF_2', 'pre_urn_Wb_2_WF_0',
       'mean_urn_y_3_w_1f0', 'mean_urn_y_2_w_0f0', 'mean_urn_y_2_w_2f0',
       'mean_urn_y_2_w_2f1', 'valuable_wk', 'number_of_persons',
       'exclusive_perc', 'non-exclusive_perc', 'N_perc', 'Y_perc',
       'Apartment_perc', 'Boat_perc', 'House_perc', 'Tentlodge_perc',
       'Unknown_perc', 'properties_count_y'],
      dtype='object')

In [82]:
# len(train_data)
train_data2=train_data.copy()
#train_data2.fillna(0,inplace=True)
print(len(train_data2))
train_data2.dropna(inplace=True)
print(len(train_data2))
train_data2.sort_values(by=['date_unique'],inplace=True)
start=-52
end=-3
weeks_day=train_data2.date_unique.unique().tolist()[start:end]
weeks_day_train=train_data2.date_unique.unique().tolist()[start-forecasting_weeks:end-forecasting_weeks]
# train_data2.drop(columns=['number_of_babies', 'number_of_pets'],inplace=True)
mean_error=[]
Allweeks_result=pd.DataFrame()
for week,weektrain in zip(weeks_day,weeks_day_train):
    train = train_data2[train_data2['date_unique'] <= weektrain]
    val = train_data2[train_data2['date_unique'] == week]
    xtr, xts = train.drop(['urn'], axis=1), val.drop(['urn'], axis=1)
    ytr, yts = train['urn'].values, val['urn'].values
    cluster_id=xts.cluster
    week1=xts.week
    pre_urn=xts.pre_urn
#    mdl =XGBRegressor(max_depth = 8 , n_estimators=90, min_child_weight=4,subsample=0.9
#                      ,colsample_bytree = 1,eval_metric = 'rmse',nthread=-1)
    mdl = LGBMRegressor(n_estimators=1000, n_jobs=-1, random_state=0)
#     lgbmcv = RFECV(estimator=mdl, step=1, cv=5)
#     lgbmcv = lgbmcv.fit(xtr,ytr)
    mdl.fit(xtr,ytr)
    p  = mdl.predict(xts)
    erro_result=pd.DataFrame({'cluster':list(cluster_id),'week':list(week1),'pre_urn':list(pre_urn),'urn':list(yts),'predicted':list(p)})
    Allweeks_result=pd.concat([Allweeks_result,erro_result])
    error = np.sqrt(metrics.mean_squared_error(yts,p))
#     print(np.corrcoef(yts,p))
#     print('Week %d - Error %.5f' % (week, error))
    mean_error.append(error)
    
print(sum(mean_error)/len(mean_error))

27136
17047
30.662313142745433


In [84]:
Allweeks_result['error']=(Allweeks_result['urn']-Allweeks_result['predicted'])/Allweeks_result['urn']
Allweeks_result['error']=abs(Allweeks_result['error'])*100

print(Allweeks_result[Allweeks_result['urn']>0].describe().head(3))
print(Allweeks_result[Allweeks_result['urn']>20].describe().head(3))
print(Allweeks_result[Allweeks_result['urn']>100].describe().head(3))
print(Allweeks_result[Allweeks_result['urn']>1000].describe().head(3))

print(np.sqrt(metrics.mean_squared_error(Allweeks_result.urn,Allweeks_result.predicted)),'mean error')
print(sum(mean_error)/len(mean_error))

           cluster         week      pre_urn          urn    predicted        error
count  3546.000000  3546.000000  3546.000000  3546.000000  3546.000000  3546.000000
mean   138.548223   27.781726    144.957981   206.550479   214.433581   28.802298  
std    67.741037    10.810968    269.743476   348.799886   349.130684   71.397723  
           cluster         week      pre_urn          urn    predicted        error
count  3004.000000  3004.000000  3004.000000  3004.000000  3004.000000  3004.000000
mean   141.580559   28.116844    169.631158   241.620506   249.696647   16.415543  
std    66.729227    10.791148    286.192630   368.191727   368.420355   18.413521  
           cluster         week      pre_urn          urn    predicted        error
count  1600.000000  1600.000000  1600.000000  1600.000000  1600.000000  1600.000000
mean   153.244375   28.795000    290.168750   408.746250   418.695871   9.207734   
std    65.331068    10.156522    349.889498   440.872979   439.522494   8.11

In [ ]:
def objective(space):
    train_data2=train_data.copy()
#     print(len(train_data2))
    train_data2.dropna(inplace=True)
#     print(len(train_data2))
    train_data2.sort_values(by=['date_unique'],inplace=True)
    start=-52
    end=-3
    weeks_day=train_data2.date_unique.unique().tolist()[start:end]
    weeks_day_train=train_data2.date_unique.unique().tolist()[start-forecasting_weeks:end-forecasting_weeks]
    # train_data2.drop(columns=['number_of_babies', 'number_of_pets'],inplace=True)
    mean_error=[]
#     print(mean_error)
    Allweeks_result=pd.DataFrame()
    for week,weektrain in zip(weeks_day,weeks_day_train):
        train = train_data2[train_data2['date_unique'] <= weektrain]
        val = train_data2[train_data2['date_unique'] == week]
        xtr, xts = train.drop(['urn'], axis=1), val.drop(['urn'], axis=1)
        ytr, yts = train['urn'].values, val['urn'].values
        cluster_id=xts.cluster
        week1=xts.week
        pre_urn=xts.pre_urn
        mdl =XGBRegressor(max_depth = int(space['max_depth']),n_estimators=200, eval_metric = 'rmse',nthread=-1)
#         mdl =XGBRegressor( colsample_bytree = float(space['colsample_bytree']),
#             learning_rate =  float(space['learning_rate']),max_depth = int(space['max_depth']),min_child_weight = int(space['min_child_weight']),subsample = float(space['subsample'])
#                           ,n_estimators= int(space['n_estimators']), eval_metric = 'rmse',nthread=-1)
    #     mdl = LGBMRegressor(n_estimators=1000, n_jobs=-1, random_state=0)
    #     lgbmcv = RFECV(estimator=mdl, step=1, cv=5)
    #     lgbmcv = lgbmcv.fit(xtr,ytr)
        mdl.fit(xtr,ytr)
        p  = mdl.predict(xts)
        erro_result=pd.DataFrame({'cluster':list(cluster_id),'week':list(week1),'pre_urn':list(pre_urn),'urn':list(yts),'predicted':list(p)})
        Allweeks_result=pd.concat([Allweeks_result,erro_result])
        error = np.sqrt(metrics.mean_squared_error(yts,p))
    #     print(np.corrcoef(yts,p))
    #     print('Week %d - Error %.5f' % (week, error))
        mean_error.append(error)
#         print(mean_error)
        loos=sum(mean_error)/len(mean_error)
        allparameter=mdl.get_params()
#         print(np.sqrt(metrics.mean_squared_error(Allweeks_result.urn,Allweeks_result.predicted)))
        
        return{'loss':loos,'space': space,'allparameter':allparameter}

    

In [ ]:
max_depth = [p for p in range(2, 10,2)]
min_child_weight=[p for p in range(3, 10,1)]
subsample=[p*0.1 for p in range(5, 10,1)]
n_estimators=[p for p in range(50, 250,40)]
colsample_bytree =[i/10.0 for i in range(4,10)]
learning_rate=[0.1,0.2,0.3,0.05,0.08]

In [ ]:
max_depth = [p for p in range(2, 10,2)]

In [ ]:
# max_depth=[8]
# min_child_weight=[4]
# subsample=[0.9]
# learning_rate=[0.1]
# colsample_bytree=[1]
# # learning_rate=[0.1,0.2,0.3,0.05,0.08]
# n_estimators=[50]


import itertools
# a = [max_depth,min_child_weight,subsample,learning_rate,n_estimators,colsample_bytree]
a=[max_depth]
aa=list(itertools.product(*a))
len(aa)

In [ ]:
result=[]
for i in aa:
    print(i)
    space={'max_depth':str(i[0])}
    reres=objective(space)
    result.append(reres)

In [ ]:
result=[]
for i in aa:
    space={'max_depth':str(i[0]),'min_child_weight':str(i[1]),'subsample':str(i[2])
           ,'learning_rate':str(i[3]),'n_estimators':i[4],'colsample_bytree':i[5]}
    reres=objective(space)
    result.append(reres)

In [ ]:
da=pd.DataFrame(result)
da.sort_values(by='loss')

In [ ]:
feat_importances = pd.Series(mdl.feature_importances_, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()

In [96]:
 df=pd.DataFrame({'cal':list(xtr.columns),'val':list(mdl.feature_importances_)})
# df.sort_values(by='val',ascending=False)
 df['Perc_imp']=(df.sort_values(by='val',ascending=False).val/ df.sort_values(by='val',ascending=False).val.sum())*100
df.sort_values(by='Perc_imp',ascending=False)

,cal,val,Perc_imp
12,date_unique,1732,5.773333
10,number_of_pets,1599,5.330000
9,number_of_babies,1591,5.303333
4,pre_urn,1547,5.156667
20,pre_urn_Wb_0_WF_2,1432,4.773333
21,pre_urn_Wb_2_WF_0,1403,4.676667
7,Indirect,1387,4.623333
18,pre_urn_Wb_0_WF_1,1284,4.280000
6,Direct,1264,4.213333
19,pre_urn_Wb_1_WF_0,1171,3.903333
